# Amazon Nova Canvas Workshop: Creating a Visual Ad for Octank Dog Food

## Introduction

Welcome to the Amazon Nova Canvas Workshop! In this hands-on session, we'll explore the powerful capabilities of Amazon Nova Canvas to create a compelling visual ad for Octank, a premium dog food company.


### Use Case

Octank is launching a new dog food line and wants to create a visual ad. They have specific requirements:

1. Generate an initial product package design for their premium dog food.
2. Create variations of the package design, including a cartoon-style version.
3. Design a special promotional package using specific brand colors.
4. Produce a professional-looking ad with the product in a kitchen setting.
5. Isolate the product image for use in various marketing materials.

### Workshop Objectives

By the end of this workshop, you will:

1. Understand the key features of Amazon Nova Canvas
2. Learn how to use these features for a real-world marketing scenario
3. Gain hands-on experience with the Amazon Bedrock API for image generation tasks

### Features We'll Use

During the workshop, we'll leverage the following features of Amazon Nova Canvas:

1. Text-to-Image: To create an initial product package design based on text description
2. Image Conditioning: To create an initial product package design inspired by the reference image 
3. Image Variation: To create a new version of the reference image by adding more details
4. Inpainting: Add more details in a specific area of the reference image
5. Color Conditioning: To generate a special promotional package design using Octank's brand color palette
6. Outpainting: To create a professional-looking ad with product in kitchen background
7. Background Removal: To isolate the product image for use in various marketing materials
8. Responsible AI - Blocked Prompts that are not appropriate

## Setup


In [ ]:
# Built-in libraries
import base64
import io
import json
import os
import sys

# External dependencies
import boto3
import botocore
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

# Set up Bedrock client
boto3_bedrock = boto3.client('bedrock-runtime')

The following ultilty function visualizes generated images alongside optional reference images. It's essential for displaying and comparing the results of image generation tasks, allowing you to easily see the input, output, and any relevant color information in a single, organized plot.

In [ ]:
# Utility function: Define plot function
def plot_images(base_images, prompt=None, seed=None, ref_image_path=None, color_codes=None, original_title=None, processed_title=None):
    if ref_image_path and color_codes:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        num_subplots = 3
    elif ref_image_path or color_codes:
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        num_subplots = 2
    else:
        fig, axes = plt.subplots(1, 1, figsize=(6, 5))
        num_subplots = 1
    
    axes = np.array(axes).ravel() 
    
    current_subplot = 0
    
    if color_codes:
        num_colors = len(color_codes)
        color_width = 0.8 / num_colors
        for i, color_code in enumerate(color_codes):
            x = i * color_width
            rect = plt.Rectangle((x, 0), color_width, 1, facecolor=f'{color_code}', edgecolor='white')
            axes[current_subplot].add_patch(rect)
        axes[current_subplot].set_xlim(0, 0.8)
        axes[current_subplot].set_ylim(0, 1)
        axes[current_subplot].set_title('Color Codes')
        axes[current_subplot].axis('off')
        current_subplot += 1
    
    if ref_image_path:
        reference_image = Image.open(ref_image_path)
        max_size = (512, 512)
        reference_image.thumbnail(max_size)
        axes[current_subplot].imshow(np.array(reference_image))
        axes[current_subplot].set_title(original_title or 'Reference Image')
        axes[current_subplot].axis('off')
        current_subplot += 1
    
    axes[current_subplot].imshow(np.array(base_images[0]))
    if processed_title:
        axes[current_subplot].set_title(processed_title)
    elif ref_image_path and seed is not None:
        axes[current_subplot].set_title(f'Image Generated Based on Reference\nSeed: {seed}')
    elif seed is not None:
        axes[current_subplot].set_title(f'Image Generated\nSeed: {seed}')
    else:
        axes[current_subplot].set_title('Processed Image')
    axes[current_subplot].axis('off')
    
    if prompt:
        print(f"Prompt: {prompt}\n")
    
    plt.tight_layout()
    plt.show()

In [ ]:
def save_image(base64_image, output_file):
    with open(output_file, 'wb') as file:
        file.write(base64.b64decode(base64_image))

## Use Cases Implementation

### Step 1: Text to Image

#### Background
Octank company wants to launch a new marketing campaign with a new set of marketing digital assets. The marketing team wants to get inspiration by leveraging image generator model to create some sample images.
As a first step they want to generate an image from simple text.

#### What are negative prompts?
Negative prompts in  Amazon Nova Canvas are a way to guide the model on what not to include in the generated image. They help refine the output by specifying elements, styles, or qualities that you want to avoid in the final image. This negative prompt below is used to improve the quality of the generated dog food package image by explicitly telling the model to avoid common issues like poor rendering, lack of detail, and unclear text.

#### What is reference_image_path ?
This is the original image (assumed to be the original picture of the owner's dog). For the purpose of this workshop, we will generate an image with Nova Canvas in the first step.



In [ ]:
# Define the prompt
prompt = "A white packet of premium dog food with an American Eskimo dog on it, professional product photography. Dog food is named Octank."
negative_prompts = "poorly rendered, poor background details, poor packet details, poor text details, bleary text"
seed = 42

# Specify path to store the output
output_save_path = "images/after_text-to-image.png" 

The Amazon Bedrock `InvokeModel` provides access to Amazon Nova Canvas by setting the right model ID, and returns a JSON response including a [Base64 encoded string](https://en.wikipedia.org/wiki/Base64) that represents the (PNG) image.

When making an `InvokeModel` request, we need to fill the `body` field with a JSON object that varies depending on the task (`taskType`) you wish to perform viz. text to image, image variation, inpainting or outpainting. The Amazon Nova Canvas models supports the following parameters:
* `cfgscale` - determines how much the final image reflects the prompt. Specifies how strongly the generated image should adhere to the prompt. Use a lower value to introduce more randomness in the generation. Min 1.1 and  Max 10. Default is 8.0.
* `seed` - a number used to initialize the generation, using the same seed with the same prompt + settings combination will produce the same results
* `numberOfImages` - the number of times the image is sampled and produced. Min - 1 and Max 5. Default 1.
* `quality` - determines the output image quality (`standard` or `premium`)

> ☝️ For more information on available input parameters for the model, refer to the [Amazon Nova User Guide](https://docs.aws.amazon.com/nova/latest/userguide/image-generation.html).

The cell below invokes the Amazon Nova Canvas model through Amazon Bedrock to create an initial image:

In [ ]:
# Generate text-to-image
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,                    # Required
            "negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 1024,        
            "width": 1024,         
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": 250 #100            # Range: 0 to 214783647
        }
    }
)

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

# save output
save_image(response_body.get("images")[0], output_save_path)

# Plot output
plot_images(response_images, processed_title="Generated Product Package") 

##### Background

Now, the marketing team wants to leverage a reference image of the company owner's American Eskimo to create a product package design.


### Step 2: Image Conditioning

Amazon Nova Canvas offers two image conditioning modes:

- Canny Edge: Extract prominent edges from the reference image to guide the generation process. You can “draw” the foundations of your desired image, and the model will then fill in the details, textures, and final aesthetic based on your guidance.

- Segmentation: Define specific regions/objects within the reference image for the model to generate content aligned with those areas.

![Visual of Canny Edge and Segementation algorithms ](https://d2908q01vomqb2.cloudfront.net/da4b9237bacccdf19c0760cab7aec4a8359010b0/2024/08/01/2024-image-generator-v2-1-color-conditioning.jpg)

Main parameters can be specified include:

- **conditionImage** (Required) – A JPEG or PNG image that guides the layout and composition of the generated image. The image must be formatted as a Base64 string. See Input Images for additional requirements.
- **controlMode** (Optional) – Specifies the conditioning mode should be used. Default value is “CANNY_EDGE”.
    
	**CANNY_EDGE** – Elements of the generated image will follow the prominent contours, or “edges”, of the condition image closely.
    
	**SEGMENTATION** – The condition image will be automatically analyzed to identify prominent content shapes. This analysis results in a segmentation mask which guides the generation, resulting in a generated image that closely follows the layout of the condition image but allows the model more freedom within the bounds of each content area.

- **controlStrength** (Optional) – Specifies how similar the layout and composition of the generated image should be to the conditioningImage. Range in 0 to 1.0 with lower values used to introduce more randomness. Default value is 0.7.

- **text** (Required) – A text prompt to generate the image. Must be 1 - 1024 characters in length.

- **negativeText** (Optional) – A text prompt to define what not to include in the image. Must be 1 - 1024 characters in length.


Note: If `controlMode` or `controlStrength` are provided, `conditionImage` must also be provided.



#### Background
The marketing team wants now to see how the package looks like with a cartoon version of the same dog.

In [ ]:
# Define the prompt, reference image
prompt = "a oil-painting dog food packet with a white american eskimo on the packet cover, dog food company name is Octank"
reference_image_path = "images/after_text-to-image.png"
seed = 42# Can be any random number between 0 to 214783647

# Specify path to store the output
output_save_path = "images/after_image_cartooning.png" 

In [ ]:
# Encode the reference image
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
    
# Generate image condition on reference image
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,  # Required
            "conditionImage": reference_image_base64, # Optional
            "controlMode": "CANNY_EDGE", # Optional: CANNY_EDGE | SEGMENTATION
            "controlStrength": 0.7,  # Range: 0.2 to 1.0,
        },
        "imageGenerationConfig": {
                "numberOfImages": 1,
                "seed": seed,
            }
        
    }
)

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

save_image(response_body.get("images")[0], output_save_path)

# plot output
plot_images(response_images, ref_image_path = reference_image_path)

### Step 3: Image Variation

#### Background


Generating images from text is powerful but, in some cases, you will want your model to understand the style from certain image and directly transfer it to your output image.
Rather than starting from scratch, image variation features enables us to do style transfer easily.

Now, Octank wants to have a dog food packet with the same style showing in the reference image, let's see how easy this step could be.

In [ ]:
# Define the prompt, reference image
prompt = "A white packet of premium dog food with an American Eskimo dog on it, professional product photography. Dog food is named Octank"
negative_prompt = "bad quality, low resolution, cartoon"
reference_image_path = "images/sketch_dog.png"
seed = 600 # Can be any random number between 0 to 214783647

# Specify path to store the output
output_save_path = "images/after_image_variation.png" 

Image variation (a.k.a. Instant Customization) – Uses 1 to 5 images and an optional prompt as input. It generates a new image that preserves the content of the input image(s), but variates its style and background.

The imageVariationParams fields are defined below.

images (Required) – A list of 1–5 images to use as references. Each must be in JPEG or PNG format and encoded as as Base64 strings. See Input Images for additional requirements.

similarityStrength (Optional) – Specifies how similar the generated image should be to the input images(s). Range in 0.2 to 1.0 with lower values used to introduce more randomness.

text (Optional) – A text prompt describing what to generate within the masked region. Must be 1 - 1024 characters in length. Omitting his field will instruct the model to remove the elements inside the masked area, replacing them with a seamless extension of the image background.

negativeText (Optional) – A text prompt to define what not to include in the image. Must be 1 - 1024 characters in length.


In [ ]:
# Encode the reference image
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

body = json.dumps({
     "taskType": "IMAGE_VARIATION",
     "imageVariationParams": {
         "text": prompt,              # Optional
         "negativeText": negative_prompt,   # Optional
         "images": [reference_image_base64],               # One image is required
        #  "similarityStrength": 1.0
     },
     "imageGenerationConfig": {
         "numberOfImages": 1,
         "quality": "premium",
         "height": 1024,
         "width": 1024,
         #"cfgScale": 10,
         "seed": seed
     }
 })

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

save_image(response_body.get("images")[0], output_save_path)

# plot output
plot_images(response_images, ref_image_path = reference_image_path)

### Step 4. Inpainting


#### Background
Octank has decided to refresh their product line by featuring different dog breeds on their packaging. However, they want to maintain consistency in the overall design and only change the dog image. This is where inpainting comes in handy. For this task, Octank wants to replace the American Eskimo dog on their current packaging with a Husky, while keeping the rest of the design intact.

Let's use inpainting to help Octank update their packaging with a new dog breed.

In [ ]:
# Define the prompt and reference image
prompt = "A white packet of premium dog food with Husky dog on it, professional product photography. Dog food is named Octank"
negative_prompts = "bad quality, low res"
reference_image_path = "images/after_image_cartooning.png" 
mask_prompt = "American Eskimo dog"
seed = 2 # Can be any random number between 0 to 214783647

`text` (Optional) – A text prompt to define what to change inside the mask. If you don't include this field, the model tries to replace the entire mask area with the background. Must be <= 512 characters. negativeText (Optional) – A text prompt to define what not to include in the image. Must be <= 512 characters. The size limits for the input image and input mask are <= 1,408 on the longer side of image. The output size is the same as the input size.

The `inPaintingParams` fields are described below. The mask defines the part of the image that you want to modify.

- `image` (Required) – The JPEG or PNG image to modify, formatted as a string that specifies a sequence of pixels, each defined in RGB values and encoded in base64. For examples of how to encode an image into base64 and decode a base64-encoded string and transform it into an image, see the code examples.
- You must define one of the following fields (but not both) in order to define.
    - `maskPrompt` – A text prompt that defines the mask.
    - `maskImage` – A string that defines the mask by specifying a sequence of pixels that is the same size as the image. Each pixel is turned into an RGB value of (0 0 0) (a pixel inside the mask) or (255 255 255) (a pixel outside the mask). For examples of how to encode an image into base64 and decode a base64-encoded string and transform it into an image, see the code examples.
- `text` (Optional) – A text prompt to define what to change inside the mask. If you don't include this field, the model tries to replace the entire mask area with the background.
- `negativeText` (Optional) – A text prompt to define what not to include in the image.

In [ ]:
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

    
# Generate image condition on reference image
body = json.dumps(
    {
        "taskType": "INPAINTING",
        "inPaintingParams": {
            "text": prompt,  # Optional - what to change inside the mask
            "negativeText": negative_prompts,    # Optional
            "image": reference_image_base64,  # Required
            "maskPrompt": mask_prompt,  # One of "maskImage" or "maskPrompt" is required
            # "maskImage": "base64-encoded string",   

        },
        "imageGenerationConfig": {
                "numberOfImages": 1,
                "seed": seed,
            }
    }
)

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

# plot output
plot_images(response_images, ref_image_path = reference_image_path)

### Step 5: Color Conditioning

#### Background
Now, let's create a special promotional package design using Octank's brand color palette.

In [ ]:
# Define the prompt, reference image, color code and path to store the generated images
prompt = "A white packet of premium dog food with an American Eskimo dog on it, professional product photography. Dog food is named Octank."
hex_color_code = ["#81FC81", "#C9D688", "#FFFFFF"]
seed = 42 # Can be any random number between 0 to 214783647

output_save_path = "images/after_color_conditioning.png"


Amazon Nova Canvas's color conditioning feature allows users to generate images that follow a specified color palette. This can be done with or without a reference image. 

Here's a summary of the parameters for color conditioning:

- colors (Required) – A list of up to 10 color codes defining the desired color palette for your image. Expressed as hexadecimal values in the form “#RRGGBB”. Examples: "#00FF00" (pure green), "#FCF2AB" (a warm yellow). The colors list has the strongest effect if a referenceImage image is not provided. Otherwise, the colors in the list and the colors from the reference image will both be used in the final output.
- referenceImage (Optional) – A JPEG or PNG image to use as a subject and style reference. The colors of the image will also be incorporated into you final output, along with the colors in from the colors list. See Input Images for additional requirements.
- text (Required) – A text prompt to generate the image. Must be 1 - 1024 characters in length.
- negativeText (Optional) – A text prompt to define what not to include in the image. Must be 1 - 1024 characters in length.



In [ ]:
# Encode the reference image
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
    
    
# Generate image condition on color palette
body = json.dumps({
    "taskType": "COLOR_GUIDED_GENERATION",
    "colorGuidedGenerationParams": {
        "text": prompt,
        "colors": hex_color_code,
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "seed": seed,
    }
})

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

save_image(response_body.get("images")[0], output_save_path)

# plot output
plot_images(response_images, color_codes = hex_color_code)

### Step 6: Outpainting

#### Background
Now, Octank wants to create a professional-looking ad with this new product with kitchen background. To do this kind of background replacement, we will use the outpainting feature offered by Nova Canvas models. 

We will first expand the image size to provide more room, then generating the new image using outpainting feature.

In [ ]:
# Define the prompt and reference image
prompt = "Dog food packet on a kitchen countertop"
reference_image_path = "images/after_color_conditioning.png" 
#reference_image_path = "images/after_image_cartooning.png" 
mask_prompt = "Dog food packet"
seed = 100 # Can be any random number between 0 to 214783647

# Expansion setting
target_width = 2048
target_height = 2048
horizontal_position_percent=0.3
vertical_position_percent=0.5

output_save_path = "images/after_outpainting.png" 

# Specify path to store the output
expand_image_path = "images/expanded_image.png"

- image (Required) – The JPEG or PNG image to modify, formatted as a Base64 string. See Input Images for additional requirements.

- You must define one of the following fields (but not both) in order to specify the area of the image to affect.

maskPrompt – A natural language text prompt that describes the region(s) of the image to edit.

maskImage – A black and white image in which pure black pixels indicate the area inside the mask and pure white pixels indicate the area outside the mask. The mask image must be the same dimensions at the input image.

- outPaintingMode – There are two outpainting modes which determine how the mask you provide is interpreted.

DEFAULT – This mode transitions smoothly between the masked area and non-masked area, using some of the pixels of the original background as the starting point for the new background. This mode is often best when you would like the new background to use similar colors as the original background near the edges of your mask, but can result in a halo effect if your prompt calls for a new background that will be very different from the original background.

PRECISE – This mode adheres strictly to the mask’s bounds and is often the best option when you are making more significant changes to the background of your image.

- text (Optional) – A text prompt describing what to generate within the masked region. Must be 1 - 1024 characters in length. Omitting his field will instruct the model to remove the elements inside the masked area, replacing them with a seamless extension of the image background.

- negativeText (Optional) – A text prompt to define what not to include in the image. Must be 1 - 1024 characters in length.

In [ ]:
# Load reference image
original_image = Image.open(reference_image_path)
original_width, original_height = original_image.size

# Calculate the position of the original image on the expanded canvas.
position = (
    int((target_width - original_width) * horizontal_position_percent),
    int((target_height - original_height) * vertical_position_percent),
)

# Create an input image which contains the original image with an expanded
# canvas.
input_image = Image.new("RGB", (target_width, target_height), (235, 235, 235))
input_image.paste(original_image, position)
input_image.save(expand_image_path)
    
# Encode the reference image
with open(expand_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

    
# Generate image condition on reference image
body = json.dumps(
    {
        "taskType": "OUTPAINTING",
        "outPaintingParams": {
            "text": prompt,  # Required
            "image": reference_image_base64,  # Required
            "maskPrompt": mask_prompt,  # One of "maskImage" or "maskPrompt" is required
            "outPaintingMode": "PRECISE",  # One of "PRECISE" or "DEFAULT"
        },
        "imageGenerationConfig": {
                "numberOfImages": 1,
                "seed": seed,
            }
        
    }
)

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

save_image(response_body.get("images")[0], output_save_path)


# plot output
plot_images(response_images, ref_image_path = reference_image_path)

### Step 7: Background Removal

#### Background
Octank has professional photos of their existing gourmet dog food. They want to use these images across various marketing materials with different background. In our last use case, we will use Background Removal feature from Amazon Nova Canvas to help Ocktank isolate its product image from their original backgrond.

To use this feature, you just need to provide the image the model needs to work with. 

In [ ]:
# Define image needs to be processed and path to store the generated images
reference_image_path = "images/after_outpainting.png" 

The background removal task type automatically identifies multiple objects in the input image and removes the background. The output image has a transparent background.

Request format
`{
    "taskType": "BACKGROUND_REMOVAL",
    "backgroundRemovalParams": {
        "image": "base64-encoded string"
    }
}`

Response Format
`{
  "images": [
    "base64-encoded string", 
    ...
  ],
  "error": "string" 
}`

The backgroundRemovalParams field is described below.
- `image` (Required) – The JPEG or PNG image to modify, formatted as a string that specifies a sequence of pixels, each defined in RGB values and encoded in base64.

In [ ]:
# Read image from file and encode it as base64 string.
with open(reference_image_path, "rb") as image_file:
    input_image = base64.b64encode(image_file.read()).decode('utf8')

body = json.dumps({
    "taskType": "BACKGROUND_REMOVAL",
    "backgroundRemovalParams": {
        "image": input_image,
    }
})

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

# plot output
plot_images(response_images, ref_image_path= reference_image_path, original_title='Original Image', processed_title='Processed Image without Background')

## Responsible AI in action

To continue supporting best practices in the responsible use of AI, Amazon Nova Canvas has built to detect and remove harmful content in the data, reject inappropriate content in the user input, and filter the models’ outputs that contain inappropriate content (such as hate speech, profanity, and violence). 

Octank marketing team wants generate an appealing campaign by placing and image of Scooby Doo on the package. 

In [ ]:
# Define the prompt with some inputs blocked for being copyright image.
prompt = "A white packet of premium dog food with Scooby Doo on it, professional product photography. Dog food is named Octank."
negative_prompts = "poorly rendered, poor background details, poor packet details, poor text details, bleary text"
seed = 42

In [ ]:
# Generate text-to-image
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,                    # Required
            "negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 1024,       
            "width": 1024,         
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": 42             # Range: 0 to 214783647
        }
    }
)

try:
    response = boto3_bedrock.invoke_model(
        body=body, 
        modelId="amazon.nova-canvas-v1:0",
        accept="application/json", 
        contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    response_images = [
        Image.open(io.BytesIO(base64.b64decode(base64_image)))
        for base64_image in response_body.get("images")
    ]

    # Plot output
    plot_images(response_images, processed_title="Generated Product Package")     

# Handle ValidationException (Responsible AI)
except boto3_bedrock.exceptions.ValidationException as error:
    print(f"An error occurred: {error}")

# Handle all the other errors
except Exception as e:
    # Handle any other unexpected exceptions
    print(f"An unexpected error occurred: {e}")


#### Amazon Bedrock gives validation error as the input prompt is asking for images of Scooby Dog and hence violates copyright.

## Summary

In this workshop, we explored the powerful features of Amazon Nova Cnavas through the lens of Octank, a premium dog food company. We covered:

- Text to Image
- Image Conditioning
- Color Variation
- Inpainting
- Color Conditioning
- Outpainting
- Background Removal
- Responsible AI in action

These tools enable Octank to efficiently create diverse, high-quality visuals for their marketing campaigns, maintaining brand consistency while adapting to various styles.

You can now leverage this GenAI-powered image generation to enhance your own creative workflows!